Import Stuff

In [6]:
import sys, time
import json
import getpass as gp
from pyfiglet import Figlet

In [7]:
def init_logo():
    f = Figlet(font='slant')
    print("---------------------------------------------------------")
    print(f.renderText(' * PAW *'))
    print(f.renderText(' Banking System '))
    print("---------------------------------------------------------")
    print(" version 1.0.0")
    print("---------------------------------------------------------")
    
def who_are_you():
    init_logo()
    with open('users.json') as f:
        user_data = json.load(f)
    user = input("Insert Card (type your user_code since we don't currently have card reader) > ")
    # if user name is not found in DB update user info
    if user not in user_data:
        print("You are a new user please provide your info")
        PIN = int(input("PIN : "))
        accounts, balance = [], []
        N = input("How many accounts? : ")

        for i in range(int(N)):
            acc = int(input("enter account id : "))
            bal = int(input("enter balance : "))
            accounts.append(acc)
            balance.append(bal)

        new_user_data = {
            user:{
                "PIN":PIN,
                "accounts":accounts,
                "balance":balance
            }
        }
        user_data.update(new_user_data)
        with open("users.json", 'w',encoding='utf-8') as f:
            json.dump(user_data, f, indent='\t')
            
    return user

In [8]:
class banking():
    def __init__(self, user):
        with open('users.json') as f:
            self.user_data = json.load(f)
        self.user = user
        print("Welcome {user}! Verify with PIN".format(user=self.user))
        self.PIN = int(gp.getpass("PIN : "))
    
    def verify_user(self):
        if self.PIN != self.user_data[self.user]["PIN"]:
            self.cnt = 0
            while self.cnt < 3:
                self.PIN = int(gp.getpass("retry PIN :"))
                if self.PIN == self.user_data[self.user]["PIN"]:
                    break
                self.cnt+=1

            if self.PIN != self.user_data[self.user]["PIN"]:
                print("wrong PIN user locked...\n Please cnatace maintanence")
                sys.exit("Killing Program")
            else:
                print("user verified")
                return True
        else:
            print("user verified")
            return True
    
    def log_in(self):
        if self.verify_user():
            return True
        else:
            sys.exit("Killing Program")
    
    def menu_page(self):
        print("\t0. Logout and Exit")
        print("\t1. View Account Balance")
        print("\t2. Deposit Cash")
        print("\t3. Withdraw Cash")
        print("\t4. Change PIN")
        print("\t5. Return Card")
        self.choice = int(input("Enter number to proceed > "))
        self.proceed_to_next()
    
    def proceed_to_next(self):
        if self.choice == 0:
            print()
            self.choose_0()
        elif self.choice == 1:
            print()
            self.choose_1()
            self.return_to_main()
        elif self.choice == 2:
            print()
            self.choose_2()
            self.return_to_main()
        elif self.choice == 3:
            print()
            self.choose_3()
            self.return_to_main()
        elif self.choice == 4:
            print()
            self.choose_4()
            self.return_to_main()
        elif self.choice == 5:
            print()
            self.choose_5()
            self.return_to_main()

    def select_account(self):
        self.L = len(self.user_data[self.user]["accounts"])
        for i in range(self.L):
            print("account: ",self.user_data[self.user]["accounts"][i])
        self.selected = int(input("select account : "))
        self.idx = self.user_data[self.user]["accounts"].index(self.selected)
        return self.selected, self.idx

    # log out done
    def choose_0(self):
        print("Exiting...")
        time.sleep(1)
        sys.exit("You have been logged out")
    
    # balance check done
    def choose_1(self):
        self.account, self.idx = self.select_account()
        print()
        print("Hello {user}! your selected account \"{account}\"".format(user=self.user, account=self.account))
        print("current balance : ${dollars}".format(dollars=self.user_data[self.user]["balance"][self.idx]))
        print()
    
    # deposit done
    def choose_2(self):
        self.account, self.idx = self.select_account()
        print()
        print("current balance is ${dollars}".format(dollars=self.user_data[self.user]["balance"][self.idx]))
        print("wait for cassette to open...") # open cassette
        time.sleep(1)
        amount = int(input("insert cash(type in the amount) : $"))
        new_balance = self.user_data[self.user]["balance"][self.idx] + amount
        print("input : ${input} , current balance : ${new}".format(input=amount, new=new_balance))
        self.user_data[self.user]["balance"][self.idx] = new_balance
        
        with open('users.json','w',encoding='utf-8') as f:
            json.dump(self.user_data, f, indent="\t")
        
        print()
    
    # withdraw done
    def choose_3(self):
        self.account, self.idx = self.select_account()
        print()
        print("current balance is ${dollars}".format(dollars=self.user_data[self.user]["balance"][self.idx]))
        amount = int(input("How much will you withdraw? : $"))
        if amount > self.user_data[self.user]["balance"][self.idx]:
            print("That exceeds your balance! retry")
            print()
            self.choose_3()
        else:
            new_balance = self.user_data[self.user]["balance"][self.idx] - amount
            self.user_data[self.user]["balance"][self.idx] = new_balance
            print("withdrawal : ${input} , current balance : ${new}".format(input=amount, new=new_balance))
            with open('users.json','w',encoding='utf-8') as f:
                json.dump(self.user_data, f, indent="\t")
        
        print()
    
    # change PIN done
    def choose_4(self):
        ans=input("Do you really want to change PIN? Y/N > ")
        if ans == "Y" or ans=="y":
            oldpin = self.user_data[self.user]["PIN"]
            newPIN=gp.getpass("Type in New PIN > ")
            confirm = gp.getpass("confirm New PIN > ")
            if newPIN == confirm:
                self.user_data[self.user]["PIN"] = int(newPIN)
                with open('users.json','w',encoding='utf-8') as f:
                    json.dump(self.user_data, f, indent="\t")
                print("PIN changed from {old} to {new}".format(old=oldpin, new=newPIN))
            else :
                confirm = gp.getpass("reconfirm New PIN > ")
                if newPIN == confirm:
                    self.user_data[self.user]["PIN"] = int(newPIN)
                    with open('users.json','w',encoding='utf-8') as f:
                        json.dump(self.user_data, f, indent="\t")
                    print("PIN changed from {old} to {new}".format(old=oldpin, new=newPIN))
                else:
                    print("PIN change cancelled")
        else :
            print("PIN change cancelled")
        print()

    # return card done
    def choose_5(self):
        ans=input("Do you really want to return card? Y/N > ")
        if ans=="Y" or ans=="y":
            print("returning card...")
            time.sleep(1)
            an=input("Will you continue with other card? Y/N > ")
            if an=="N" or an=="n":
                sys.exit("program exit")
            elif an=="Y" or an=="y":
                who_are_you()
        else:
            print("Return card cancelled")
        print()

    def return_to_main(self):
        self.next = input("Do you want to proceed to menu Y/N > ")
        if self.next=="Y" or self.next=="y":
            self.menu_page()
        else:
            sys.exit("program exit")


In [9]:
def main():
    user = who_are_you()
    bank_use = banking(user)
    if bank_use.log_in():
        bank_use.menu_page()

In [11]:
main()

---------------------------------------------------------
               ____  ___ _       __       
      __/|_   / __ \/   | |     / /  __/|_
     |    /  / /_/ / /| | | /| / /  |    /
    /_ __|  / ____/ ___ | |/ |/ /  /_ __| 
     |/    /_/   /_/  |_|__/|__/    |/    
                                          

       ____              __   _            
      / __ )____ _____  / /__(_)___  ____ _
     / __  / __ `/ __ \/ //_/ / __ \/ __ `/
    / /_/ / /_/ / / / / ,< / / / / / /_/ / 
   /_____/\__,_/_/ /_/_/|_/_/_/ /_/\__, /  
                                  /____/   
   _____            __                    
  / ___/__  _______/ /____  ____ ___      
  \__ \/ / / / ___/ __/ _ \/ __ `__ \     
 ___/ / /_/ (__  ) /_/  __/ / / / / /     
/____/\__, /____/\__/\___/_/ /_/ /_/      
     /____/                               

---------------------------------------------------------
 version 1.0.0
---------------------------------------------------------
Welcome mjay! Verify with PIN

ValueError: invalid literal for int() with base 10: ''